# Train Model

In [170]:
import sys
sys.path.append("..")
%reload_ext autoreload
%autoreload 2

In [171]:
import pandas as pd
from sklearn_pandas import cross_val_score, DataFrameMapper
from pandas import Categorical
from sklearn.ensemble import RandomForestClassifier

In [172]:
from ares.models import featurize
from ares.utils import load_wp10

In [173]:
input_file = "../data/enwiki.labeling_revisions.w_features.nettrom_30k.csv.gz"
revisions = load_wp10(input_file)

# Create a pipeline

In [174]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer, PolynomialFeatures

In [175]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegressionCV
from sklearn.base import BaseEstimator, TransformerMixin

sqrt_cols = ['words',
             'headings',
             'sub_headings',
             'images',
             'categories',
             'wikilinks',

             'who_templates',
             'main_templates',
             'cite_templates',
             # infobox as a binary
             'citation_needed',
             'other_templates',

             'ref',
             'smartlists',
             # coordinates
             'coordinates']

binarized_cols = ['coordinates', 'infoboxes']

mapper = DataFrameMapper([
    (sqrt_cols, FunctionTransformer(func=np.sqrt)),
    (binarized_cols, FunctionTransformer(func=lambda x: x.astype(bool)))
])

clf = LogisticRegressionCV(multi_class='multinomial', 
                           random_state=1234, 
                           penalty='l2',
                           fit_intercept=True, 
                           n_jobs=-1)
clf = RandomForestClassifier(bootstrap=True, oob_score=True)

In [176]:
pipe = Pipeline([
    ('mapper', mapper),
    ('poly', PolynomialFeatures(degree=2)),
    ('PCA', PCA(whiten=True)),
    ('clf', clf)
])

In [201]:
# Create a single session instance to reuse
session = mwapi.Session(host='https://en.wikipedia.org',
                        user_agent='wikiedits-app <jeffrey.arnold@gmail.com>')




# Fit Model on Full Sample

In [177]:
categories = list(revisions['wp10'].dtype.categories)

In [178]:
fitted = pipe.fit(X=revisions.copy(), y=revisions['wp10'])

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


In [179]:
with open("../data/classifier-1.pkl", "wb")  as f:
    dill.dump(fitted, f)

In [204]:
def get_current_page(session, title):
    r = session.get(action="query", titles=title, prop="revisions",
                    rvprop="ids|content|timestamp", rvslots="main")
    page = list(r['query']['pages'].values())[0]
    # There is no such page!
    if 'missing' in page:
        return None
    rev = page['revisions'][0]
    rev['content'] = rev['slots']['main']['*']
    del rev['slots']
    rev['title'] = page['title']
    rev['pageid'] = page['pageid']
    return rev


# Evaluate Model Peformance

In [205]:
session = mwapi.Session("https://en.wikipedia.org", user_agent="<jeffrey.arnold@gmail.com>")

In [206]:
r = get_current_page(session, "Winston Churchill")

In [207]:
r

{'revid': 861295866,
 'parentid': 861295754,
 'timestamp': '2018-09-26T12:34:56Z',
 'content': '{{Redirect|Churchill||Churchill (disambiguation)|and|Winston Churchill (disambiguation)}}\n{{pp-semi|small=yes}}\n{{pp-move-indef}}\n{{Use British English|date=August 2010}}\n{{Use dmy dates|date=January 2018}}\n{{Infobox officeholder\n|name             = Winston Churchill\n|office           = [[Prime Minister of the United Kingdom]]\n|honorific-prefix = [[The Right Honourable]] [[Sir]]\n|honorific-suffix = {{post-nominals|country=GBR|size=100%|sep=|KG|OM|CH|TD|DL|FRS|RA}}\n|image            = Sir Winston Churchill - 19086236948.jpg\n|alt              = Churchill wearing a suit, standing and holding a chair\n|caption          = Winston Churchill in the [[Centre Block|Canadian Parliament]], December 1941 by [[Yousuf Karsh]]\n|deputy           = Anthony Eden\n|monarch          = {{ubl|[[George VI]]|[[Elizabeth II]]}}\n|predecessor      = [[Clement Attlee]]\n|successor        = [[Anthony Eden]]

# TODO

1. Cross validate
2. Out of sample
3. Other